# ***DISTRIBUCIÓN DE SEGMENTOS***

### DataFrames:

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "queretaro" in nombre or 'df_junio_2023_queretaro' in nombre
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro',
 'df_junio_2023_queretaro']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Categoria':'categoria'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'categoria']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

### CONTEO ÚNICO DE CATEGORÍAS

In [4]:
conteo_total = {}
for df in dataframes_list:
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,S1,10577
1,S2,4506
2,S3,3009
3,,2440
4,B2,2586
5,A3,2264
6,B1,2040
7,A1,1974
8,B3,2107
9,E3,1107


In [5]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,10577,S,1
1,4506,S,2
2,3009,S,3
3,2440,NaN,1
4,2586,B,2
5,2264,A,3
6,2040,B,1
7,1974,A,1
8,2107,B,3
9,1107,E,3


In [6]:
#segmentos_dist = conteo_total_df.groupby(['segmento', 'clase']) # Agrupar
segmentos_dist = conteo_total_dfs.groupby(['segmento', 'clase'])['conteo_total'].sum()
segmentos_dist

segmento  clase
A         1         1974
          2         1859
          3         2264
B         1         2040
          2         2586
          3         2107
C         1         1259
          2         1110
          3         1188
D         1          692
          2          872
          3          747
E         1         1071
          2          682
          3         1107
L         1         1177
          2          234
          3          349
S         1        13199
          2         4506
          3         3009
Name: conteo_total, dtype: int64

## Porcentajes totales de registros por segmento

In [7]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  44032


,segmento,total_por_segmento,porcentaje_del_total
0,A,6097,13.846748
1,B,6733,15.291152
2,C,3557,8.078216
3,D,2311,5.248456
4,E,2860,6.495276
5,L,1760,3.997093
6,S,20714,47.043060


In [8]:
#Obtener el total de registros por segmento y clase
total_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum()
#Calcular los porcentajes
segmentos_percent = conteo_total_df.merge(total_segmento, on='segmento', suffixes=('', '_total'))
segmentos_percent['porcentaje'] = segmentos_percent['conteo_total'] / segmentos_percent['conteo_total_total'] * 100
#Seleccionar las columnas necesarias y eliminar las columnas auxiliares
segmentos_percent = segmentos_percent[['segmento', 'clase', 'porcentaje']]
segmentos_percent

,segmento,clase,porcentaje
0,S,1,51.062084
1,S,2,21.753403
2,S,3,14.526407
3,B,2,38.407842
4,A,3,37.133016
5,B,1,30.298530
6,A,1,32.376579
7,B,3,31.293628
8,E,3,38.706294
9,A,2,30.490405


Porcentaje de cada clase en relación al % que representa su segmento del total.

In [9]:
temp_dfs = [] # Almacenar los DataFrames temporales de cada segmento
# Realizamos un bucle sobre cada segmento
for segmento in resultados['segmento'].unique():
    # Filtramos las clases correspondientes al segmento actual
    clases_segmento = segmentos_percent[segmentos_percent['segmento'] == segmento]
    
    # Calculamos el total de porcentaje del segmento actual
    total_porcentaje_segmento = resultados[resultados['segmento'] == segmento]['porcentaje_del_total'].iloc[0]
    
    # Calculamos el porcentaje que representa cada clase en relación al porcentaje del segmento
    porcentaje_clase = (clases_segmento['porcentaje'] / 100) * total_porcentaje_segmento
    
    # DataFrame temporal para el segmento actual
    temp_df = pd.DataFrame({
        'segmento': [segmento] * len(clases_segmento),
        'clase': clases_segmento['clase'].values,
        'porcentaje_clase': porcentaje_clase#.round()
    })
    
    # Agrega DataFrame temporal a la lista
    temp_dfs.append(temp_df)

# Concatena todos los DataFrames temporales en un solo DataFrame
porcentajes_clase = pd.concat(temp_dfs, ignore_index=True)
porcentajes_clase

,segmento,clase,porcentaje_clase
0,A,3,5.141715
1,A,1,4.483103
2,A,2,4.221930
3,B,2,5.873001
4,B,1,4.632994
5,B,3,4.785156
6,C,3,2.698038
7,C,1,2.859284
8,C,2,2.520894
9,D,2,1.980378


In [10]:
# Agrupar por 'segmento' y 'clase', y sumar los porcentajes
porcentajes_clase_agrupado = porcentajes_clase.groupby(['segmento', 'clase']).agg({'porcentaje_clase': 'sum'}).reset_index()

# Luego aplicar pivot
porcentajes_clase_ordenado = porcentajes_clase_agrupado.pivot(index='segmento', columns='clase', values='porcentaje_clase')
porcentajes_clase_ordenado.fillna(0, inplace=True)
porcentajes_clase_ordenado

clase,1,2,3
segmento,,,
A,4.483103,4.221930,5.141715
B,4.632994,5.873001,4.785156
C,2.859284,2.520894,2.698038
D,1.571584,1.980378,1.696493
E,2.432322,1.548874,2.514081
L,2.673056,0.531432,0.792605
S,29.975927,10.233467,6.833666


## Gráfica

In [11]:
segmentos = list(porcentajes_clase_ordenado.index)
segmentos_sorted = ['E','D','C','B','A','S','L']  # Orden deseado
clases = porcentajes_clase_ordenado.columns # Clases como etiquetas de las series de datos
clases = ['1', '2', '3']  # Clases en el orden deseado
# Asegúrate de que todos los segmentos mencionados estén en los datos con valor 0 si no existen
for segmento in segmentos_sorted:
    if segmento not in porcentajes_clase_ordenado.index:
        porcentajes_clase_ordenado.loc[segmento] = [0] * len(clases)

# Reordenar el DataFrame para seguir el orden deseado en `segmentos_sorted`
porcentajes_clase_ordenado = porcentajes_clase_ordenado.loc[segmentos_sorted]
colores = {
    '1': '#2962ff',
    '2': '#9500ff',
    '3': '#ff0059'
}
# Almacenar las barras de cada clase
barras = []
for clase in clases:
    barras.append(go.Bar(
        y=segmentos_sorted,  # Eje y: Segmentos
        x=porcentajes_clase_ordenado[clase],  # Eje x: Porcentajes de clase
        name=f'{clase}',  # Nombre de la serie de datos
        orientation='h',  # Orientación horizontal
        text=porcentajes_clase_ordenado[clase].apply(lambda x: f'{x:.2f}%'),  # Texto con los porcentajes
        textposition='auto',  # Posición automática del texto
        marker_color=colores[clase]   # Color de las barras
    ))

fig = go.Figure(data=barras)
fig.update_layout(
    yaxis=dict(
        title='Segmento', # Título del eje y        
    ),
    legend_title="Clase",
    barmode='stack',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico      
    xaxis=dict(
        title='Porcentaje', # Título del eje x
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
    ),  
    margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)
)

# Función para guardar la gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_dist_segmentos', carpeta='assets\graficas')

fig.show()